# Neighbourhoods in Toronto

Importing required libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from geopy.geocoders import Nominatim

Getting the web page.

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')

In [3]:
#print(soup.prettify())

Setting column names

In [4]:
table = soup.find('table', class_ = 'wikitable sortable')
tabtr = table.find('tr')
col=[]
for i in tabtr.find_all('th'):
    col.append(i.text) 
col[2] = col[2][0:-1]
col[0] = "PostalCode"

Getting the data in form of lists

In [5]:
body = table.find('tbody')
a=0
pc_list = []
bor_list = []
neb_list = []

for i in body.find_all('tr'):
    for j in i.find_all('td'):
        if (a%3)==0:
            #print(a%3,j.text)
            pc_list.append(j.text)
            a+=1
        elif (a%3)==1:
            #print(a%3,j.text)
            bor_list.append(j.text)
            a+=1
        elif (a%3)==2:
            #print(a%3,j.text)
            neb_list.append(j.text[0:-1])
            a+=1
#print(pc_list)
#print(bor_list)
#print(neb_list)

In [6]:
data = {col[0]:pc_list,col[1]:bor_list,col[2]:neb_list}
#data
df = pd.DataFrame(data, columns = col)
#df

In [7]:
df_drop = df.drop(df[df.Borough == "Not assigned"].index)
#df_drop

In [8]:
df_drop.loc[df_drop.Neighbourhood == "Not assigned", 'Neighbourhood'] = "Queen's Park"


In [9]:
df_drop2 = df_drop
df_drop2 = df_drop2.groupby('PostalCode').agg({"PostalCode": lambda x:max(x), "Neighbourhood": lambda x: set(x), "Borough": lambda x:max(x)})
df_drop2 = df_drop2[['PostalCode','Borough','Neighbourhood']]
df_drop2.columns

Index(['PostalCode', 'Borough', 'Neighbourhood'], dtype='object')

In [10]:
df_lats = pd.read_csv(r'C:\Users\Rishab\Desktop\Study Subjects\IBM Data Science\capstone\Geospatial_Coordinates.csv')
df_lats.shape

(103, 3)

In [11]:
df_drop2.sort_values(by=['PostalCode'])
df_lats.sort_values(by=['PostalCode'])
df_lats.shape

C:\Users\Rishab\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 'PostalCode' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  """Entry point for launching an IPython kernel.


(103, 3)

In [12]:
df_drop2 = df_drop2.reset_index(drop=True)
df_drop2['Neighbourhood'] = df_drop2['Neighbourhood'].astype(str)
df_list = df_drop2['Neighbourhood']
for i in range(0,len(df_list)):
    df_list[i] = df_list[i][1:len(df_list[i])-1]

In [13]:
df_joined = pd.DataFrame({'PostalCode':df_drop2["PostalCode"],'Borough':df_drop2["Borough"],'Neighbourhood':df_list,'Latitude':df_lats["Latitude"],'Longitude':df_lats["Longitude"]})

In [14]:
#df_final = df_joined
#df_final['Neighbourhood'] = df_final['Neighbourhood'].astype(str)
#print(df_final.Neighbourhood)
#type(df_final)

In [15]:
#df_list = 
#print(df_final["Neighbourhood"][0])
#df_list[0] = df_list[0][1:len(df_list[0])-1]
#print(df_list[0])
#for i in range(0,len(df_list)):
#    df_list[i] = df_list[i][1:len(df_list[0])-1]
#print(df_list[0])

In [17]:
df_joined[['PostalCode','Borough','Neighbourhood','Latitude','Longitude']]
df_joined.head()

,Borough,Latitude,Longitude,Neighbourhood,PostalCode
0,Scarborough,43.806686,-79.194353,"'Rouge', 'Malvern'",M1B
1,Scarborough,43.784535,-79.160497,"'Highland Creek', 'Rouge Hill', 'Port Union'",M1C
2,Scarborough,43.763573,-79.188711,"'Guildwood', 'Morningside', 'West Hill'",M1E
3,Scarborough,43.770992,-79.216917,'Woburn',M1G
4,Scarborough,43.773136,-79.239476,'Cedarbrae',M1H


In [24]:
#x = df_joined["Borough"].str.find("Toronto")
df_try = df_joined.where((df_joined["Borough"].str.find("Toronto"))>=0)
df_final = df_try.dropna()

In [25]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(len(df_final['Borough'].unique()),df_final.shape[0]))

The dataframe has 4 boroughs and 39 neighborhoods.


In [29]:

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [30]:
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium 

In [32]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_final['Latitude'], df_final['Longitude'], df_final['Borough'], df_final['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto
#,parse_html=False

In [33]:
VERSION = '20180605' # Foursquare API version
CLIENT_ID = 'HSEHOEWSINXPTMIHL0FCVNNVWZEJIWS54SKYWLKHBJM1KE1E' # your Foursquare ID
CLIENT_SECRET = 'E1C0YZ3BTGF24QH2RVP5K2YP5UDZEN2PUSH5LE0Z4YI5RMME' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HSEHOEWSINXPTMIHL0FCVNNVWZEJIWS54SKYWLKHBJM1KE1E
CLIENT_SECRET:E1C0YZ3BTGF24QH2RVP5K2YP5UDZEN2PUSH5LE0Z4YI5RMME


In [34]:
from pandas.io.json import json_normalize

In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [36]:
toronto_venues = getNearbyVenues(names=df_final['Neighbourhood'],
                                   latitudes=df_final['Latitude'],
                                   longitudes=df_final['Longitude'])

'The Beaches'
'The Danforth West', 'Riverdale'
'India Bazaar', 'The Beaches West'
'Studio District'
'Lawrence Park'
'Davisville North'
'North Toronto West'
'Davisville'
'Moore Park', 'Summerhill East'
'South Hill', 'Rathnelly', 'Summerhill West', 'Forest Hill SE', 'Deer Park'
'Rosedale'
'Cabbagetown', 'St. James Town'
'Church and Wellesley'
'Harbourfront'
'Ryerson', 'Garden District'
'St. James Town'
'Berczy Park'
'Central Bay Street'
'Adelaide', 'Richmond', 'King'
'Union Station', 'Harbourfront East', 'Toronto Islands'
'Toronto Dominion Centre', 'Design Exchange'
'Commerce Court', 'Victoria Hotel'
'Roselawn'
'Forest Hill West', 'Forest Hill North'
'North Midtown', 'The Annex', 'Yorkville'
'University of Toronto', 'Harbord'
'Chinatown', 'Grange Park', 'Kensington Market'
'South Niagara', 'Railway Lands', 'Bathurst Quay', 'CN Tower', 'King and Spadina', 'Harbourfront West', 'Island airport'
'Stn A PO Boxes 25 The Esplanade'
'First Canadian Place', 'Underground city'
'Christie'
'Dufferin

In [40]:
print(toronto_venues.shape)
toronto_venues.head()
toronto_venues.groupby('Neighborhood').count()
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

(869, 7)
There are 191 uniques categories.
